In [1]:
from socket import socket
import time

In [64]:
ip = "********"
port = 8080

def request_response(request,ip,port):
    s = socket()
    s.connect((ip,port))
    s.sendall(request)
    time.sleep(2)
    print(s.recv(4096).decode())

### TEST 1

200 Expect ; Simple Valid Request ; also check for content-length, which should be 307

In [34]:
test1 =b"GET /index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test1,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>



### TEST 2

200 Expect ; Support Subdir ; /subdir1/index.html 

In [35]:
test2 =b"GET /subdir1/index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test2,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 177

<html>
<head>
    <title>Basic subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>



### TEST 3

200 Expect ; translate / to /index.html

In [41]:
test3 = b"GET / HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test3,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>



### Test 4

200 Expect ; translate /subdir1/ to /subdir/index.html

In [42]:
test4 =b"GET /subdir1/ HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test4,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 177

<html>
<head>
    <title>Basic subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>



### Test 5

400 Expect ; malform GET 

In [43]:
test5 =b"get /index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test5,ip,port)

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0




### Test 6

400 Expect ; malform HTTP/1.1 

In [45]:
test6 =b"GET /index.html http\r\nHost: Ha\r\n\r\n"
request_response(test6,ip,port)

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0




### Test 7

400 Expect ; no host in request

In [47]:
test7 =b"GET /index.html HTTP/1.1\r\n\r\n"
request_response(test7,ip,port)

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0




### Test 8 

200 Expect ; test connection close header

In [49]:
test8 =b"GET /index.html HTTP/1.1\r\nHost: Ha\r\nConnection: close\r\n\r\n"
request_response(test8,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307
Connection: close

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>



### Test 9

404 Expect ; test file not found

In [52]:
test9 =b"GET /sue.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test9,ip,port)

HTTP/1.1 404 Not Found
Server: Go-Triton-Server-1.0




### Test 10

400 Expect ; missing forward slash

In [54]:
test10 =b"GET index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test10,ip,port)

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0




### Test 11

404 Expect ; file escaping doc root

In [69]:
test11 =b"GET /../../request.input HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test11,ip,port)

HTTP/1.1 404 Not Found
Server: Go-Triton-Server-1.0




### Test 12
200 Expect; file not escaping doc root

In [7]:
test12 =b"GET /subdir1/subdir11/../../subdir1/subdir11 HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test12,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Monday, 02-Nov-20 00:10:11 UTC
Content-Type: text/html
Content-Length: 184

<html>
<head>
    <title>Twice subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a double-subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>



### Test 13

404 Expect ; file not found in subdir

In [71]:
test13 =b"GET /subdir1/sue.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test13,ip,port)

HTTP/1.1 404 Not Found
Server: Go-Triton-Server-1.0




### Test 14

400 Expect ; malformed header

In [73]:
test14 =b"GET /index.html HTTP/1.1\r\nHost Ha\r\n\r\n"
request_response(test14,ip,port)

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0




### Test 15

200 Expect ; other header added

In [77]:
test15 =b"GET /index.html HTTP/1.1\r\nHost: Ha\r\nAgent: Chrome\r\n\r\n"
request_response(test15,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>



### Test 16

400 Expect ; Partial Request + Timeout

In [83]:
s = socket()
s.connect((ip,port))
test16 =b"GET /index.html HTTP/1.1\r\n"
s.sendall(test16)
start = time.time()
print(s.recv(4096).decode())
end = time.time()
print('time interval in seconds: ',end - start)

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0


time interval in seconds:  6.196410179138184


### Test 17

200, 200, 200 Expect ; Valid Pipeline

In [84]:
test17 =b"GET /index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
test17 +=b"GET /subdir1/index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
test17 +=b"GET /subdir1/subdir11/index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test17,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>
HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 177

<html>
<head>
    <title>Basic subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>
HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 184

<html>
<head>
    <title>Twice subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a

### Test 18

200, 404, 200 Expect ; Pipeline + 404

In [85]:
test18 =b"GET /index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
test18 +=b"GET /hello.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
test18 +=b"GET /subdir1/subdir11/index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test18,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>
HTTP/1.1 404 Not Found
Server: Go-Triton-Server-1.0

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 184

<html>
<head>
    <title>Twice subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a double-subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>



### Test 19

200, 400 Expect ; Pipleline + malform

In [88]:
test19 =b"GET /subdir1/index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
test19 +=b"GET /index.html HTTP/1.1\r\nHost:: Ha\r\n\r\n"
request_response(test19,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 177

<html>
<head>
    <title>Basic subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>
HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0




### Test 20

200,200, N/A ; Pipleline + connection close

In [90]:
test20 =b"GET /index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
test20 +=b"GET /subdir1/index.html HTTP/1.1\r\nHost: Ha\r\nConnection: close\r\n\r\n"
test20 +=b"GET /subdir1/subdir11/index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test20,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>
HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: text/html
Content-Length: 177
Connection: close

<html>
<head>
    <title>Basic subdirectory file</title>
</head>
<body>
<h1>This is a basic index file for a subdirectory</h1>
You can use this for testing.
<p>
</body>
</html>



### Test 21

200 Expect ; get image file

In [93]:
test21 =b"GET /UCSD_Seal.png HTTP/1.1\r\nHost: Ha\r\n\r\n"
s = socket()
s.connect((ip,port))
s.sendall(test21)
time.sleep(2)
print(s.recv(4096))

b'HTTP/1.1 200 OK\r\nServer: Go-Triton-Server-1.0\r\nLast-Modified: Friday, 23-Oct-20 09:38:02 UTC\r\nContent-Type: image/png\r\nContent-Length: 637155\r\n\r\n\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x07\x80\x00\x00\x07\x80\x08\x06\x00\x00\x00/\xa2J\xc6\x00\x00\x00\x06bKGD\x00\xff\x00\xff\x00\xff\xa0\xbd\xa7\x93\x00\x00 \x00IDATx\x9c\xec\xddw\x9c\\\xe5y\xf7\xff\xef}\xa6\xefl\xafZ\xedjU\x91\x90\x04\xa2H\x98"\x81\x01\x17J\x00\x938\xe0\xd8I\x9cb\xc7\xd8\x01\xcb-\x0e\xc9/O~V\xf28qb;\x14\t\'&\x89\x83;1`cLs\x03aP\x03$\x04\x08\x15\x84z\xd9^gwv\xfa\xb9\x9f?$0\x180H\x9a\xdd3\xe5\xf3~\xbd\xf4\x92v\xc5\xde\xe7\xbbB\x9a\xb9\xe7\\s]\xb7\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

### Test 22

200 Expect ; File type outside MimeMap

In [96]:
test22 =b"GET /subdir1/subdir11/maoyo.giaogiao HTTP/1.1\r\nHost: Ha\r\n\r\n"
request_response(test22,ip,port)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Friday, 23-Oct-20 09:38:02 UTC
Content-Type: application/octet-stream
Content-Length: 4

Yeet


### Test 23
200, 400, 404 Expect; Concurrency test

In [22]:
def concurrency_test(request):
    s = socket()
    s.connect((ip,port))
    s.sendall(request)
    time.sleep(2)
    print(s.recv(4096).decode())
    print()

In [25]:
from multiprocessing import Pool
pool = Pool(processes=3)
test23 =[b"GET / HTTP/1.1\r\nHost: Ha\r\n\r\n", 
         b"GET /subdir1/sue.html HTTP/1.1\r\nHost: Ha\r\n\r\n", 
         b"GET index.html HTTP/1.1\r\nHost: Ha\r\n\r\n"]
results = pool.map(request_response, test23)

HTTP/1.1 200 OK
Server: Go-Triton-Server-1.0
Last-Modified: Monday, 02-Nov-20 00:10:11 UTC
Content-Type: text/html
Content-Length: 307

<html>
<head>
<title>Basic index file</title>
</head>
<body>
<h1>This is a basic index file</h1>
You can use this for testing.
<p>
<ul>
<li><a href=UCSD_Seal.png alt="UCSD Seal">UCSD seal</a>
<li><a href=kitten.jpg alt="Kitten">Kitten photo</a>
<li><a href=subdir1/>A subdirectory</a>
</ul>
</body>
</html>

HTTP/1.1 400 Bad Request
Server: Go-Triton-Server-1.0


HTTP/1.1 404 Not Found
Server: Go-Triton-Server-1.0


